### Python code to explore rich-lists for tokens

In [1]:
# !pip install pandas
# !pip install tqdm

In [2]:
import pandas as pd
import requests
from tqdm.notebook import tqdm
pd.set_option('display.float_format', lambda x: '%.1f' % x)

token_to_analyze = 'kushti' #choose a token to analyze

In [3]:
token_ids = {'kushti':'fbbaac7337d051c10fc3da0ccb864f4d32d40027551e1c3ea3ce361f39b91e40',
             'erdoge':'36aba4b4a97b65be491cf9f5ca57b5408b0da8d0194f30ec8330d1e8946161c1',
             'lunadog':'5a34d53ca483924b9a6aa0c771f11888881b516a8d1a9cdc535d063fe26d065e',
             'sigusd':'03faf2cb329f2e90d6d23b58d91bbb6c046aa143261cc21f52fbe2824bfcbf04',
             }

token_id = token_ids[token_to_analyze]

api_token = 'https://api.ergoplatform.com/api/v1/boxes/unspent/byTokenId/'
explorer_url = 'https://explorer.ergoplatform.com/en/token/'+token_id
api_token_info = 'https://api.ergoplatform.com/api/v1/tokens/'

token_info = requests.get(api_token_info+token_id).json()
token_name = token_info['name']
total_supply = token_info['emissionAmount']
print(f"Token name: {token_name} -- emission amount: {total_supply} (decimals: {token_info['decimals']})")

Token name: kushti -- emission amount: 1000000000 (decimals: 0)


In [4]:
req_string = api_token+token_id

token_df = pd.DataFrame()
limit = 100

def amount_in_box(assets,token_id):
    token_assets = [i for i in assets if i['tokenId']==token_id]
    token_assets = token_assets[0]
    token_amount = token_assets['amount']
    return token_amount

for p in tqdm(range(0,1000)):
    req = requests.get(f'{req_string}?offset={str(p*limit)}&limit={str(limit)}')
    req_df = pd.DataFrame(req.json()['items'])
    if len(req_df)==0:
        break
    token_df_page = pd.DataFrame()
    token_df_page['address'] = req_df['address']
    token_df_page['amount'] = req_df['assets'].apply(lambda x: amount_in_box(x,token_id))
    token_df_page['id'] = req_df['boxId']
    token_df = pd.concat([token_df,token_df_page])
    token_df = token_df.drop_duplicates(keep='first')
    supply_counted = 100*token_df['amount'].sum()/total_supply
    if p%10 == 0:
        print(f"[Page {p}]: Boxes scanned = {token_df.shape[0]} -- supply counted = {supply_counted :.2f}% ")
        
#sum up the boxes in each address
token_df = token_df.groupby(['address'],as_index=False).sum()
token_df = token_df.sort_values(by='amount',ascending=False).reset_index().drop(columns='index')
total_accounted = token_df['amount'].sum()
token_df['percentOfTotal'] = token_df['amount'].apply(lambda x: 100*x/total_supply)
token_df['amount'] = token_df['amount'].apply(lambda x: x*(10**(-token_info['decimals'])))
print(f"Finished scanning - counted {100*total_accounted/total_supply :.2f}% of total_supply")

  0%|          | 0/1000 [00:00<?, ?it/s]

[Page 0]: Boxes scanned = 100 -- supply counted = 30.78% 
[Page 10]: Boxes scanned = 1100 -- supply counted = 48.60% 
[Page 20]: Boxes scanned = 2100 -- supply counted = 56.71% 
[Page 30]: Boxes scanned = 3100 -- supply counted = 56.81% 
[Page 40]: Boxes scanned = 4100 -- supply counted = 57.12% 
[Page 50]: Boxes scanned = 5100 -- supply counted = 59.06% 
[Page 60]: Boxes scanned = 6100 -- supply counted = 59.96% 
[Page 70]: Boxes scanned = 7100 -- supply counted = 71.95% 
[Page 80]: Boxes scanned = 8100 -- supply counted = 86.63% 
Finished scanning - counted 99.92% of total_supply


In [5]:
#display top 20
token_df.head(20)

,address,amount,percentOfTotal
0,3gb1RZucekcRdda82TSNS4FZSREhGLoi1FxGDmMZdVeLtY...,247856077,24.8
1,9gRqfAe8nsdT9FweH8U4SeyHP75op95qrmrxT2TwPkAjUd...,98800000,9.9
2,9h16dKnYsi5e7YeiRbnKfkj2y5HzbAGJDcUMDZduCHbfmc...,60330790,6.0
3,9ghcSLzTUe9c7czPShdvpvCiB5z3GMxuo4HQJv73VttGUw...,57080844,5.7
4,9fznGPQqEHaFPfkAEuvWYtDNetSW7bRLJMmWUAzMZnrqLB...,54000000,5.4
5,9hqgwPRXRYsMst1AgNNUXmgQu8vkPGGtEoxfAcBwmyTNAp...,46960279,4.7
6,9guJBqnUSaLvpX3Dj6ee6J4hxEJwPQUkV3HdtQp5cbPXGG...,45597941,4.6
7,9iCznZztuQARpWj86P53QFkN6UhyHXGUuRAxD9EtacfVUt...,34155019,3.4
8,9g1sK9dcAcxkqTRC864Ldpp89bXGxNaxYyVgAj9j5hGV3w...,25000000,2.5
9,9hXN8UhqZ55zsuJGENs9a3wS8cEKz2rpe2aZpSeU92HUaL...,25000000,2.5
